Parser for basis sets from the [Basis Set Exchange](https://www.basissetexchange.org/) in JSON format

In [ ]:
import json

example_data = {
    "molssi_bse_schema": {
        "schema_type": "complete",
        "schema_version": "0.1"
    },
    "revision_description": "Data from Gaussian09",
    "revision_date": "2018-06-19",
    "elements": {
        "1": {
            "electron_shells": [
                {
                    "function_type": "gto",
                    "region": "",
                    "angular_momentum": [
                        0
                    ],
                    "exponents": [
                        "0.3425250914E+01",
                        "0.6239137298E+00",
                        "0.1688554040E+00"
                    ],
                    "coefficients": [
                        [
                            "0.1543289673E+00",
                            "0.5353281423E+00",
                            "0.4446345422E+00"
                        ]
                    ]
                }
            ],
            "references": [
                {
                    "reference_description": "STO-3G Minimal Basis (3 functions/AO)",
                    "reference_keys": [
                        "hehre1969a"
                    ]
                }
            ]
        },
        "8": {
            "electron_shells": [
                {
                    "function_type": "gto",
                    "region": "",
                    "angular_momentum": [
                        0
                    ],
                    "exponents": [
                        "0.1307093214E+03",
                        "0.2380886605E+02",
                        "0.6443608313E+01"
                    ],
                    "coefficients": [
                        [
                            "0.1543289673E+00",
                            "0.5353281423E+00",
                            "0.4446345422E+00"
                        ]
                    ]
                },
                {
                    "function_type": "gto",
                    "region": "",
                    "angular_momentum": [
                        0,
                        1
                    ],
                    "exponents": [
                        "0.5033151319E+01",
                        "0.1169596125E+01",
                        "0.3803889600E+00"
                    ],
                    "coefficients": [
                        [
                            "-0.9996722919E-01",
                            "0.3995128261E+00",
                            "0.7001154689E+00"
                        ],
                        [
                            "0.1559162750E+00",
                            "0.6076837186E+00",
                            "0.3919573931E+00"
                        ]
                    ]
                }
            ],
            "references": [
                {
                    "reference_description": "STO-3G Minimal Basis (3 functions/AO)",
                    "reference_keys": [
                        "hehre1969a"
                    ]
                }
            ]
        }
    },
    "version": "1",
    "function_types": [
        "gto"
    ],
    "names": [
        "STO-3G"
    ],
    "tags": [],
    "family": "sto",
    "description": "STO-3G Minimal Basis (3 functions/AO)",
    "role": "orbital",
    "auxiliaries": {},
    "name": "STO-3G"
}


